# Data Mining
- 將時間詞彙替換成同義詞彙並產生新的對話

In [1]:
import os
import io
import re
from random import choice
import numpy as np
import pandas as pd

import baseline
from utils import load_raw_data
from data_generator import generate_dataset

In [2]:
articles, labeling_df = load_raw_data('data/train_2/train_2.txt')

In [3]:
# read wordnet
# wordnet = pd.read_excel('data/WordNet.xlsx', encoding='utf-8')
wordnet_file = pd.ExcelFile('data/WordNet.xlsx')
wn1 = wordnet_file.parse(wordnet_file.sheet_names[0])
wn2 = wordnet_file.parse(wordnet_file.sheet_names[1])

## Sheet 1

In [ ]:
wn1.dropna(how='all', inplace=True)
wn1 = wn1.astype(str)
for i, row in wn1.iterrows():
    wn1.loc[i, '可替換詞彙'] = wn1.loc[i, '目標詞彙'] + '/' + '/'.join([w for w in row[2:] if w != 'nan'])   
wn1.drop(wn1.columns[3:], axis=1, inplace=True)
wn1.reset_index(drop=True, inplace=True)
wn1

In [ ]:
wn1.loc[6, '可替換詞彙'] = '/'.join([alt for alt in wn1.loc[6, '可替換詞彙'].split('/') if alt not in ['宵', '夕']])
wn1.loc[18, '可替換詞彙'] = '/'.join(['學校', '校'])
wn1.loc[28, '可替換詞彙'] = '/'.join(['弟弟', '弟'])
wn1.loc[35, '可替換詞彙'] = '/'.join([alt for alt in wn1.loc[35, '可替換詞彙'].split('/') if alt != '郎'])
wn1.drop([3, 11, 16, 41], inplace=True)
wn1.reset_index(drop=True, inplace=True)

In [ ]:
# 將訓練集中的有標籤的詞彙，替換成隨機一種同義詞彙，產生新的訓練集接在原訓練集後面
# 是否能夠確保每個同義詞的詞頻都不會差太遠
wn1_gbt = wn1.groupby('類型')
alt_dict = {}
for tag in wn1_gbt.groups:
    alt_set = list(map(lambda x: x.split('/'), wn1_gbt.get_group(tag)['可替換詞彙'].tolist()))
    alt_dict[tag] = alt_set
alt_dict

## Sheet 2

In [ ]:
wn2.drop(['類別'], axis=1, inplace=True)
type_dict = dict(wn2)
for tag in type_dict.keys():
    type_dict[tag] = [token for token in type_dict[tag].tolist() if token is not np.nan]
type_dict

## 替換新詞

In [ ]:
for tag in alt_dict.keys():
    df = labeling_df[labeling_df['entity_type'] == tag]
    for alt_list in alt_dict[tag]:
        for i, row in df.iterrows():
            for alt in alt_list:
                if alt in row['entity_text']:
                    start_pos = row['start_position'] + row['entity_text'].find(alt[0])
                    article_id = row['article_id']
                    article = articles[article_id]
                    for_choice = [token for token in alt_list if token != alt]
                    new_token = choice(for_choice)
                    end_pos = start_pos+len(new_token)
                    new_article = article[:start_pos] + new_token + article[end_pos:]
                    articles[article_id] = new_article
                    labeling_df.loc[i, ['end_position', 'entity_text']] = [end_pos, new_article[row['start_position']:end_pos]]
                    break

In [ ]:
# 將新的訓練集寫入檔案
with open('data/train_2/train_2_new.txt', mode='w+', encoding='utf-8') as f:
    for i in range(len(articles)):
        f.write(articles[i]+'\n')
        ios = io.StringIO()
        labeling_df[labeling_df['article_id'] == i].to_csv(ios, index=False, encoding='utf-8', sep='\t')
        f.write(ios.getvalue().replace('\r', ''))
        f.write('\n--------------------\n\n')

## Number Mask

In [9]:
labeling_df.loc[3831:, 'article_id'] += 200

In [17]:
zhnums = list('零一二三四五六七八九十')
for i in range(len(articles)):
    articles[i] = re.sub(f'(\d|{"|".join(zhnums)})', 'N', articles[i])

labeling_df.loc[:, 'entity_text'] = labeling_df['entity_text'].apply(lambda text: re.sub(f'(\d|{"|".join(zhnums)})', 'N', text))

In [18]:
labeling_df

,article_id,start_position,end_position,entity_text,entity_type
0,0,69,71,前天,time
1,0,75,77,前天,time
2,0,738,740,NN,med_exam
3,0,741,744,NNN,med_exam
4,0,809,811,前年,time
...,...,...,...,...,...
7657,399,36,39,N.N,med_exam
7658,399,40,43,N.N,med_exam
7659,399,48,51,N.N,med_exam
7660,399,53,56,N.N,med_exam


In [19]:
with open('data/train_2/train_2_nmask.txt', mode='w+', encoding='utf-8') as f:
    for i in range(len(articles)):
        f.write(articles[i]+'\n')
        ios = io.StringIO()
        labeling_df[labeling_df['article_id'] == i].to_csv(ios, index=False, encoding='utf-8', sep='\t')
        f.write(ios.getvalue().replace('\r', ''))
        f.write('\n--------------------\n\n')